In [1]:
import numpy as np
from stl import mesh
from PIL import Image

np.set_printoptions(precision=3, suppress=True)

#import and adjust image
mirror_img = Image.open('burst.png').convert("L")
src_a = np.asarray(mirror_img) / 255.0 # normalized grayscale
src_a = 1.0 - src_a #invert colors for convenience
gray_range = src_a.max() - src_a.min() #find range of values
src_a = (src_a - src_a.min()) / (gray_range) #stretch contrast 0.0 to 1.0
nw, nh = src_a.shape

<frozen importlib._bootstrap>:488: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


In [2]:
#define hexagon grid centers
W = 10
H = 9
r = 11.5 #distance from center of hex to a flat
#x centers are 2*r apart, starting at r
Xctr_v = np.array(range(W)) * 2*r + r
#y centers have shorter spacing by sqrt(3)/2 because of the staggered offset
sqt3 = 3**0.5
Yctr_v = np.array(range(H)) * r*sqt3 + 2*r/sqt3
#create a grid of coordinate pairs from these vectors
#ctr_a = np.array([[(a,b) for a in Xctr_v] for b in Yctr_v])
Xctr_a, Yctr_a = np.meshgrid(Xctr_v,Yctr_v)
ctr_a = np.dstack((Xctr_a, Yctr_a))
#stagger the rows, every other row is shifted x+0.5
for i, row in enumerate(ctr_a[:,:,0]):
    for j, c in enumerate(row):
        if i%2:
            ctr_a[i,j,0] += r

In [3]:
# map grid x position to heading direction
x_range = np.max(ctr_a[:,:,0]) + r # find range of x ctr coordinates
hdg_a = 2*ctr_a[:,:,0]/x_range - 1.0 # create normalized heading array
#hdg_a = hdg_a**3 * np.pi/12 # use a cubic rolloff shape -15 to +15 degrees
hdg_a = np.tanh(hdg_a)/np.tanh(1) * np.pi/12 # use a tanh rolloff shape -15 to +15 degrees
# add small random noise to headings
np.random.seed(42)
noise_strength = np.pi/120 # 1.5 degrees stdev
wiggler = np.random.default_rng()
hdg_a = hdg_a + wiggler.normal(0.0, noise_strength, size=(H, W))

In [4]:
#subsample the image under each hexagon and average the pixel values
elv_a = hdg_a*0.0 #initialize the elevation array
#find the size of one hexagon, approximated by an equal-area square
sh = sw = r * (3**0.25) / (2**0.5) #half the side of the square, physical size
#find the total height of the hexagon array
ah = ctr_a[-1,0,1] + 2*r/sqt3
#find the total width of the hexagon array
aw = ctr_a[1,-1,0] + r
#scale the 'square size' of each hexagon to the image pixel space
sw = int(round(sw/aw*nw,0))
sh = int(round(sh/ah*nh,0))
#step through hex center locations, using y position matrix
for i, row in enumerate(ctr_a[:,:,1]):
    for j, y in enumerate(row):
        x = ctr_a[i,j,0] # pull matching x coordinate
        Y = int(round(y/(ah)*nh,0)) # translate this coord to image space
        X = int(round(x/(aw)*nw,0))
        # average a cropped copy of the image in this hex's region
        elv_a[i,j] = src_a[(X-sw):(X+sw),(Y-sh):(Y+sh)].mean(axis=0).mean()
        
#grayscale to elevation angle (0=lowest, 1=highest)
gamma = 1.5
min_elv = -np.pi/30 # -6 degrees min elevation
max_elv =  np.pi/6  # 30 degrees max elevation
elv_a = (elv_a ** gamma) * (max_elv - min_elv) + min_elv
#elv_a = -elv_a #reverse rotation directions (right hand rule)

In [5]:
# generate base hexagon points, as rows in an array
th = np.pi/3 # 60 degrees
rz60 = np.array([[np.cos(th), np.sin(th),0],[-np.sin(th), np.cos(th),0],[0,0,1]])
temp = np.array([r, r/sqt3, 0])
pt0_a = np.empty((6,3))
for i in range(6):
    pt0_a[i,:] = temp
    temp = temp@rz60

In [6]:
# generate a larger array of scaled, rotated, and positioned hex points
hex_a = np.empty((H,W,6,3))
for i, row in enumerate(ctr_a[:,:,0]):
    for j, x in enumerate(row):
        y = ctr_a[i,j,1]
        hd = hdg_a[i,j]
        el = elv_a[i,j]
        r3h = np.array([[np.cos(hd),0,-np.sin(hd)],[0,1,0],[np.sin(hd),0,np.cos(hd)]])
        r3e = np.array([[1,0,0],[0,np.cos(el),-np.sin(el)],[0,np.sin(el),np.cos(el)]])
        hex_a[i,j,:,:] = ((11.0/11.5)*pt0_a)@r3e@r3h + [x,y,0]

In [7]:
# create a vector of bottom edge points
be_v = ctr_a[0,:,:]-[r,2*r/sqt3+r/2]
be_v[0,0] = -r/2
be_v = np.vstack((be_v,[aw+r/2,-r/2]))
be_v = np.hstack((be_v,np.zeros((W+1,1))))
# create a vector of left edge points
le_v = ctr_a[:,0,:]-[0,3*r/2/sqt3]
le_v[:,0] = -r/2
le_v[0,1] = -r/2
le_v = np.vstack((le_v,[-r/2,ah+r/2]))
le_v = np.hstack((le_v,np.zeros((H+1,1))))
#create a vector of top edge points
te_v = be_v + [0,ah+r,0]
te_v[1:-1,0] += r
#create a vector of right edge points
re_v = le_v + [aw+r,0,0]

In [58]:
tri_a = []
#create main mirror face
for i, row in enumerate(hex_a):
    for j, HA in enumerate(row):
        # main face, 4 triangles
        tri_a.append([HA[0,:],HA[2,:],HA[3,:]])
        tri_a.append([HA[0,:],HA[3,:],HA[5,:]])
        tri_a.append([HA[0,:],HA[1,:],HA[2,:]])
        tri_a.append([HA[3,:],HA[4,:],HA[5,:]])
        # grout on top edge of each triangle
        if i<(H-1):
            if not (i%2==0 and j==0):
                tri_a.append([ HA[2,:], HA[1,:], hex_a[i+1,j+(i%2-1),5,:] ])
                tri_a.append([ HA[2,:], hex_a[i+1,j+(i%2-1),5,:], hex_a[i+1,j+(i%2-1),4,:] ])
            if not (i%2==1 and j==(W-1)):
                tri_a.append([ HA[1,:], HA[0,:], hex_a[i+1,j+(i%2),3,:] ])
                tri_a.append([ HA[0,:], hex_a[i+1,j+(i%2),4,:], hex_a[i+1,j+(i%2),3,:] ])
        # grout on right side of each triangle:
        if j<(W-1):
            tri_a.append([ HA[0,:], HA[5,:], hex_a[i,j+1,2,:] ])
            tri_a.append([ HA[5,:], hex_a[i,j+1,3,:], hex_a[i,j+1,2,:] ])
            if not i==(H-1):
                tri_a.append([ HA[0,:], hex_a[i,j+1,2,:], hex_a[i+1,j+(i%2),4,:] ])
            if not i==0:
                tri_a.append([ HA[5,:], hex_a[i-1,j+(i%2),1,:], hex_a[i,j+1,3,:] ])
        # bottom skirt
        if i==0:
            tri_a.append([ HA[3,:], be_v[j], HA[4,:] ])
            tri_a.append([ HA[4,:], be_v[j+1], HA[5,:] ])
            tri_a.append([ HA[4,:], be_v[j], be_v[j+1] ])
            if j>0:
                tri_a.append([ HA[3,:], hex_a[i,j-1,5,:], be_v[j] ])
        # top skirt
        elif i==(H-1):
            tri_a.append([ HA[2,:], HA[1,:], te_v[j] ])
            tri_a.append([ HA[1,:], HA[0,:], te_v[j+1] ])
            tri_a.append([ HA[1,:], te_v[j+1], te_v[j] ])
            if j>0:
                tri_a.append([ HA[2,:], te_v[j], hex_a[i,j-1,0,:] ])
        # left skirt
        if j==0:
            tri_a.append([ HA[3,:], le_v[i+1], le_v[i] ])
            tri_a.append([ HA[2,:], le_v[i+1], HA[3,:] ])
            if i>0:
                tri_a.append([ HA[3,:], le_v[i], hex_a[i-1,j,2,:] ])
                if not i%2:
                    tri_a.append([ HA[4,:], HA[3,:], hex_a[i-1,j,2,:] ])
            if i<(H-1) and not i%2:
                tri_a.append([ HA[2,:], HA[1,:], hex_a[i+1,j,3,:] ])
        # right skirt
        elif j==(W-1):
            tri_a.append([ HA[5,:], re_v[i], re_v[i+1] ])
            tri_a.append([ HA[0,:], HA[5,:], re_v[i+1] ])
            if i<(H-1):
                tri_a.append([ HA[0,:], re_v[i+1], hex_a[i+1,j,5,:] ])
                if i%2:
                    tri_a.append([ HA[1,:], HA[0,:], hex_a[i+1,j,5,:] ])
            if i>0 and i%2:
                tri_a.append([ HA[5,:], HA[4,:], hex_a[i-1,j,0,:] ])

In [59]:
# create back face
thk = ah/6
C = [aw/2, ah/2, -thk/2]
slope = [[0,0,(2-d/(len(le_v)-1))*thk/2] for d in range(len(le_v))]
vbak = [[0,0,3*thk/2-((1-2*d/(len(be_v)-1))**2)*thk/2] for d in range(len(be_v))]
for i in range(len(be_v)-1):
    tri_a.append([ C, be_v[i]-vbak[i], be_v[i+1]-vbak[i+1] ])
    tri_a.append([ C, te_v[i]-[0,0,thk/2], te_v[i+1]-[0,0,thk/2] ])
for i in range(len(le_v)-1):
    tri_a.append([ C, le_v[i]-slope[i], le_v[i+1]-slope[i+1] ])
    tri_a.append([ C, re_v[i]-slope[i], re_v[i+1]-slope[i+1] ])

In [60]:
# create four sides
# bottom
t1 = np.array([ be_v[:-1], be_v[:-1]-vbak[:-1], be_v[1:] ]).transpose(1,0,2)
t2 = np.array([ be_v[1:], be_v[:-1]-vbak[:-1], be_v[1:]-vbak[1:] ]).transpose(1,0,2)
# top
t3 = np.array([ te_v[:-1], te_v[1:], te_v[:-1]-[0,0,thk/2] ]).transpose(1,0,2)
t4 = np.array([ te_v[1:], te_v[1:]-[0,0,thk/2], te_v[:-1]-[0,0,thk/2] ]).transpose(1,0,2)
# ...and stack
tri_a = np.vstack((tri_a,t1,t2,t3,t4))
# left
t1 = np.array([ le_v[:-1], le_v[1:], le_v[:-1]-slope[:-1] ]).transpose(1,0,2)
t2 = np.array([ le_v[1:], le_v[1:]-slope[1:], le_v[:-1]-slope[:-1] ]).transpose(1,0,2)
# right
t3 = np.array([ re_v[:-1], re_v[:-1]-slope[:-1], re_v[1:] ]).transpose(1,0,2)
t4 = np.array([ re_v[1:], re_v[:-1]-slope[:-1], re_v[1:]-slope[1:] ]).transpose(1,0,2)
# ...and stack
tri_a = np.vstack((tri_a,t1,t2,t3,t4))

In [61]:
# convert to numpy array
faces_array = np.asarray(tri_a)

# Create mesh
surf_mesh = mesh.Mesh(np.zeros(faces_array.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces_array):
    surf_mesh.vectors[i] = f

# Export STL
surf_mesh.save('/home/morgans/Documents/hex_mirror.stl')